<a href="https://colab.research.google.com/github/kiratkJaura/Final_Project03/blob/main/Final_Project03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Change directory to your project folder in Google Drive
import os
project_dir = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data'  #PATH to Project3 Data file
os.chdir(project_dir)

# Install required libraries
!pip install opencv-python-headless ultralytics matplotlib scikit-learn

# Import libraries
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

##  Step 1: Object Masking
image_path = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/motherboard_image.JPEG'  # PATH to Motherboard_image
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresholded = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
)
edges = cv2.Canny(thresholded, 100, 200)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mask = np.zeros_like(gray)
cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
extracted = cv2.bitwise_and(image, image, mask=mask)
extracted_path = 'extracted_motherboard.JPEG'
cv2.imwrite(extracted_path, extracted)
print(f"Extracted image saved to {extracted_path}")

##  Step 2: YOLOv8 Training
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml'  #PATH to data.yaml to run YOLOv8
model = YOLO('yolov8n.pt')
model.train(data=dataset_path, epochs=50, imgsz=900, batch=16, name='pcb_model')  #Defining epochs, imagesize, batch and name of the model
model_path = f"{project_dir}/runs/detect/pcb_model"
print(f"Model saved at {model_path}")

##  Step 3: Evaluate the Model
trained_model = YOLO(model_path + '/weights/best.pt')
test_images = [
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/ardmega.jpg',    #PATH to ardmega
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/arduno.jpg',     #PATH to arduno
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/rasppi.jpg'      #PATH to rasppi
]

# Analyze and save results
for img in test_images:
    results = trained_model.predict(img, save=True, save_dir=f"{model_path}/evaluation_results")
    print(f"Results for {img} saved to {model_path}/evaluation_results")

##  Step 4: Extract Ground Truth Labels (y_true)
labels_dir ='/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/test/labels' #PATH to Test_labels
test_image_names = ['evaluation_image1', 'evaluation_image2', 'evaluation_image3']

y_true = []
for image_name in test_image_names:
    label_file = os.path.join(labels_dir, f"{image_name}.txt")
    with open(label_file, 'r') as file:
        labels = [line.split()[0] for line in file.readlines()]
        y_true.extend(labels)
y_true = list(map(int, y_true))
print("Ground truth labels (y_true):", y_true)

# # Step 5: Extract Predicted Labels (y_pred)
y_pred = []
for img in test_images:
    results = trained_model.predict(img)
    predicted_classes = [int(box.cls) for box in results[0].boxes]
    y_pred.extend(predicted_classes)
print("Predicted labels (y_pred):", y_pred)

## Step 6: Generate and Visualize Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='viridis')
plt.title("Confusion Matrix")
plt.show()

##  Step 7: Visualize Training Metrics
metrics = trained_model.val()
precision = metrics['precision']
recall = metrics['recall']
plt.plot(recall, precision, label="Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

# Additional Suggestions for Report
print("Key insights:")
print("- Use extracted image and saved evaluation results for the report.")
print("- Include precision-recall curve and confusion matrix in your submission.")


Extracted image saved to extracted_motherboard.JPEG
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=900, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pcb_model2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

train: Scanning /content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/train/labels.cache... 43 images, 501 backgrounds, 0 corrupt: 100%|██████████| 544/544 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/valid/labels.cache... 0 images, 8 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/pcb_model2/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 928 train, 928 val
Using 0 dataloader workers
Logging results to runs/detect/pcb_model2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.623      41.47      1.115        143        928:  44%|████▍     | 15/34 [10:20<12:45, 40.29s/it]